# Test relation between frequency and Vo

## Imports

In [1]:
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.layouts import column, row
from bokeh.models import Range1d
from bokeh.io import export_png
output_notebook()

import numpy as np

from voltagebudget.neurons import adex
from voltagebudget.util import burst
from voltagebudget.util import find_phis
from voltagebudget.util import read_stim
from voltagebudget.util import read_modes
from voltagebudget.util import create_times

Loading BokehJS ...

/Users/type/anaconda/envs/py2/lib/python2.7/site-packages/brian2/core/variables.py:174: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  return np.issubdtype(np.bool, self.dtype)


## Fns

In [2]:
def plot_voltages(x, E, tspan=(0.2, 0.24), random=0, N=10, dt=1e-5):
    times = create_times(tspan, dt)
    neurons = list(range(N))
    if random > 0:
        np.random.shuffle(neurons)
        neurons = neurons[0:random]

    p = figure(plot_width=600, plot_height=300)

    
    
    p.line(x=E, y=np.linspace(x.min(), x.max(), 10), color="purple")
    p.line(x=times, y=x.mean(1), color='red', alpha=0.8)
    for n in neurons:
        p.line(x=times, y=x[:, n], color="black", alpha=0.2)

    p.xaxis.axis_label = 'Time (s)'
    p.yaxis.axis_label = 'V (volts)'
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    p.x_range = Range1d(*tspan)
    show(p)

# Shared parameters

In [3]:
t = 0.4
d = -2e-3
E = 0.22
w = 2e-3
f = 4
A = 0.1e-9
n_cycles = 3
time_step = 1e-5
N = 500

stim = "../data/stim1.csv"
mode = 'regular'
time_step = 1e-5
seed_value = 42

params, w_in, bias_in, _ = read_modes(mode)

# Read stim

In [4]:
stim_data = read_stim(stim)
ns_x = np.asarray(stim_data['ns'])
ts_x = np.asarray(stim_data['ts'])

# Est phis

In [5]:
phi_w, phi_E = find_phis(E, f, d, verbose=False)

# Generate phi_E network

In [6]:
tau_m = 0.02

ns_E, ts_E, voltages_E = adex(
    N,
    t,
    ns_x,
    ts_x,
    E=E, #+ d - (2.5 * tau_m),
    n_cycles=n_cycles,
    w_in=w_in,
    bias_in=bias_in,
    f=f,
    A=A,
    phi=phi_E,
    sigma=0,
    budget=True,
    save_args=None,
    time_step=time_step,
    seed_value=seed_value,
    **params)

In [ ]:
plot_voltages(voltages_E['V_m'].T, E=E+d, tspan=(0, t), random=20)

In [ ]:
plot_voltages((np.absolute(voltages_E['V_osc'].T) - 0.005), E=E+d, tspan=(0, t), random=20)

# Pulse at E+d

In [7]:
tau_m = 20e-3
E_adj = E + d - (1 * tau_m)
pulse_params =(A, E_adj, E_adj + (1 * tau_m))

ns_w, ts_w, voltages_w = adex(
    2,
    t,
    ns_x,
    ts_x,
    E=E_adj,
    n_cycles=0,
    w_in=w_in,
    bias_in=bias_in,
    f=0,
    A=0,
    phi=0,
    sigma=0,
    budget=True,
    save_args=None,
    time_step=time_step,
    pulse_params=pulse_params,
    seed_value=seed_value,
    **params)

In [9]:
plot_voltages((np.absolute(voltages_w['V_osc'].T) - 0.005), E=E+d, tspan=(0, t), N=2, random=2)